In [70]:
import pandas as pd
import numpy as np
import json
import re

with open('plays.json') as data_file:    
    json_data = json.load(data_file)  


In [71]:
plays_json = pd.json_normalize(json_data)
plays_json.head()

,period,homeScore,scoringPlay,priority,statYardage,awayScore,wallclock,modified,id,text,...,expectedPoints.added,winProbability.before,winProbability.after,winProbability.added,start.shortDownDistanceText,start.possessionText,start.downDistanceText,scoringType.displayName,scoringType.name,scoringType.abbreviation
0,1,0,False,False,0,0,2020-12-21T19:34:38Z,2020-12-21T19:35Z,401256135101849902,Zach Williams kickoff for 60 yds,...,-2.823851,0.000100,0.075747,0.075647,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0,False,False,-2,0,2020-12-21T19:36:09Z,2020-12-21T19:36Z,401256135101855901,Camerun Peoples run for a loss of 2 yards to t...,...,-1.372518,0.918380,0.898592,-0.019788,1st & 10,APP 35,1st & 10 at APP 35,NaN,NaN,NaN
2,1,0,False,False,0,0,2020-12-21T19:36:29Z,2020-12-21T19:36Z,401256135101858101,Zac Thomas pass incomplete to Jalen Virgil,...,-0.958074,0.898592,0.883065,-0.015527,2nd & 12,APP 33,2nd & 12 at APP 33,NaN,NaN,NaN
3,1,0,False,False,4,0,2020-12-21T19:37:31Z,2020-12-21T19:37Z,401256135101864601,Zac Thomas run for 4 yds to the AppSt 37,...,-0.664023,0.883065,0.879816,-0.003248,3rd & 12,APP 33,3rd & 12 at APP 33,NaN,NaN,NaN
4,1,0,False,False,0,0,2020-12-21T19:38:31Z,2020-12-21T19:38Z,401256135101867201,"Xavier Subotsch punt for 29 yds, downed at the...",...,-0.878193,0.879816,0.880597,0.000781,4th & 8,APP 37,4th & 8 at APP 37,NaN,NaN,NaN


In [72]:
plays_json.columns

Index(['period', 'homeScore', 'scoringPlay', 'priority', 'statYardage',
       'awayScore', 'wallclock', 'modified', 'id', 'text', 'playType',
       'start.distance', 'start.yardLine', 'start.team.id', 'start.down',
       'start.yardsToEndzone', 'start.posTeamTimeouts',
       'start.defTeamTimeouts', 'clock.displayValue', 'type.id', 'type.text',
       'type.abbreviation', 'end.shortDownDistanceText', 'end.possessionText',
       'end.downDistanceText', 'end.distance', 'end.yardLine', 'end.team.id',
       'end.down', 'end.yardsToEndzone', 'end.posTeamTimeouts',
       'end.defTeamTimeouts', 'expectedPoints.before', 'expectedPoints.after',
       'expectedPoints.added', 'winProbability.before', 'winProbability.after',
       'winProbability.added', 'start.shortDownDistanceText',
       'start.possessionText', 'start.downDistanceText',
       'scoringType.displayName', 'scoringType.name',
       'scoringType.abbreviation'],
      dtype='object')

In [117]:
##--Play type vectors------
scores_vec = [
"Blocked Punt Touchdown",
"Blocked Punt (Safety)",
"Punt (Safety)",
"Blocked Field Goal Touchdown",
"Missed Field Goal Return Touchdown",
"Fumble Recovery (Opponent) Touchdown",
"Fumble Return Touchdown",
"Interception Return Touchdown",
"Pass Interception Return Touchdown",
"Punt Touchdown",
"Punt Return Touchdown",
"Sack Touchdown",
"Uncategorized Touchdown",
"Defensive 2pt Conversion",
"Uncategorized",
"Two Point Rush",
"Safety",
"Penalty (Safety)",
"Punt Team Fumble Recovery Touchdown",
"Kickoff Team Fumble Recovery Touchdown",
"Kickoff (Safety)",
"Passing Touchdown",
"Rushing Touchdown",
"Field Goal Good",
"Pass Reception Touchdown",
"Fumble Recovery (Own) Touchdown"
]

defense_score_vec = [
"Blocked Punt Touchdown",
"Blocked Field Goal Touchdown",
"Missed Field Goal Return Touchdown",
"Punt Return Touchdown",
"Fumble Recovery (Opponent) Touchdown",
"Fumble Return Touchdown",
"Kickoff Return Touchdown",
"Defensive 2pt Conversion",
"Safety",
"Sack Touchdown",
"Interception Return Touchdown",
"Pass Interception Return Touchdown",
"Uncategorized Touchdown"
]
turnover_vec = [
  "Blocked Field Goal",
"Blocked Field Goal Touchdown",
"Blocked Punt",
"Blocked Punt Touchdown",
"Field Goal Missed",
"Missed Field Goal Return",
"Missed Field Goal Return Touchdown",
"Fumble Recovery (Opponent)",
"Fumble Recovery (Opponent) Touchdown",
"Fumble Return Touchdown",
"Defensive 2pt Conversion",
"Interception",
"Interception Return",
"Interception Return Touchdown",
"Pass Interception Return",
"Pass Interception Return Touchdown",
"Kickoff Team Fumble Recovery",
"Kickoff Team Fumble Recovery Touchdown",
"Punt Touchdown",
"Punt Return Touchdown",
"Sack Touchdown",
"Uncategorized Touchdown"
]
normalplay = [
"Rush",
"Pass",
"Pass Reception",
"Pass Incompletion",
"Pass Completion",
"Sack",
"Fumble Recovery (Own)"
]
penalty = [
'Penalty', 
'Penalty (Kickoff)',
'Penalty (Safety)'
]
offense_score_vec = [
"Passing Touchdown",
"Rushing Touchdown",
"Field Goal Good",
"Pass Reception Touchdown",
"Fumble Recovery (Own) Touchdown",
"Punt Touchdown", #<--- Punting Team recovers the return team fumble and scores
"Punt Team Fumble Recovery Touchdown",
"Kickoff Touchdown", #<--- Kickoff Team recovers the return team fumble and scores
"Kickoff Team Fumble Recovery Touchdown"
]
punt_vec = [
"Blocked Punt",
"Blocked Punt Touchdown",
"Blocked Punt (Safety)",
"Punt (Safety)",
"Punt",
"Punt Touchdown",
"Punt Team Fumble Recovery",
"Punt Team Fumble Recovery Touchdown",
"Punt Return Touchdown"
]
kickoff_vec = [
"Kickoff",
"Kickoff Return (Offense)",
"Kickoff Return Touchdown",
"Kickoff Touchdown",
"Kickoff Team Fumble Recovery",
"Kickoff Team Fumble Recovery Touchdown",
"Kickoff (Safety)",
"Penalty (Kickoff)"
]
int_vec = [
"Interception",
"Interception Return",
"Interception Return Touchdown",
"Pass Interception",
"Pass Interception Return",
"Pass Interception Return Touchdown"
]

def clean_pbp_dat(play_df):
    play_df.id = play_df.id.astype(int)
    play_df = play_df.sort_values(by="id", ascending=True)
    #-- Touchdowns----
    play_df["scoring_play"] = np.where(play_df["type.text"].isin(scores_vec), 1, 0)
    play_df["td_play"] = play_df.text.str.match("touchdown|for a TD", case=False, flags=0, na=False)
    play_df["touchdown"] = play_df["type.text"].str.match("touchdown", case=False, flags=0, na=False)
    play_df["safety"] = play_df["text"].str.match("safety", case=False, flags=0, na=False)

#       #-- Fumbles----
    play_df["fumble_vec"] = play_df["text"].str.match("fumble", case=False, flags=0, na=False)
    
#       #-- Kicks----
    play_df["kickoff_play"] = np.where(play_df["type.text"].isin(kickoff_vec), 1, 0)
    play_df["kickoff_tb"] = np.where(play_df["text"].str.match("touchback", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_onside"] = np.where(play_df["text"].str.match("on-side|onside|on side", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_oob"] = np.where(play_df["text"].str.match("out-of-bounds|out of bounds", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_fair_catch"] = np.where(play_df["text"].str.match("fair catch|fair caught", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kickoff_downed"] = np.where(play_df["text"].str.match("downed", case=False, flags=0, na=False) & play_df.kickoff_play == True, 1, 0)
    play_df["kick_play"] = play_df["text"].str.match("kick|kickoff", case=False, flags=0, na=False)
    play_df["kickoff_safety"] = np.where(~play_df["type.text"].isin(['Blocked Punt','Penalty']) & play_df["text"].str.match("kickoff", case=False, flags=0, na=False) & play_df.safety == True, 1, 0)
    
#       #-- Punts----
    play_df["punt"] = np.where(play_df["type.text"].isin(punt_vec), 1, 0)
    play_df["punt_play"] = play_df["text"].str.match("punt", case=False, flags=0, na=False)
    play_df["punt_tb"] = np.where(play_df["text"].str.match("touchback", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["punt_oob"] = np.where(play_df["text"].str.match("out-of-bounds|out of bounds", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["punt_fair_catch"] = np.where(play_df["text"].str.match("fair catch|fair caught", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["punt_downed"] = np.where(play_df["text"].str.match("downed", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)
    play_df["kickoff_safety"] = np.where(play_df["type.text"].isin(['Blocked Punt','Punt']) & play_df["text"].str.match("punt", case=False, flags=0, na=False) & play_df.safety == True, 1, 0)
    play_df["penalty_safety"] = np.where(play_df["type.text"].isin(['Penalty']) & play_df.safety == True, 1, 0)
    play_df["punt_blocked"] = np.where(play_df["text"].str.match("blocked", case=False, flags=0, na=False) & play_df.punt == True, 1, 0)

#       #-- Pass/Rush----
    play_df['rush'] = np.where(
        ((play_df["type.text"] == "Rush")
        | (play_df["type.text"] == "Rushing Touchdown")
        | (play_df["type.text"].isin(["Safety","Fumble Recovery (Opponent)","Fumble Recovery (Opponent) Touchdown", "Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Return Touchdown"]) & play_df["text"].str.match("run for", case=False, flags=0, na=False))),
        1, 0
    )
    
    play_df['pass'] = np.where(
        (
        (play_df["type.text"].isin(["Pass Reception", "Pass Completion","Pass Touchdown","Sack","Pass","Inteception","Pass Interception Return", "Interception Return Touchdown","Pass Incompletion","Sack Touchdown"]))
        | ((play_df["type.text"] == "Safety") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
        | ((play_df["type.text"] == "Safety") & play_df["text"].str.match("pass complete", case=False, flags=0, na=False))
         | ((play_df["type.text"] == "Fumble Recovery (Own)") & play_df["text"].str.match("pass complete|pass incomplete|pass intercepted", case=False, flags=0, na=False))
         | ((play_df["type.text"] == "Fumble Recovery (Own)") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
         | ((play_df["type.text"] == "Fumble Recovery (Own) Touchdown") & play_df["text"].str.match("pass complete|pass incomplete|pass intercepted", case=False, flags=0, na=False))
          | ((play_df["type.text"] == "Fumble Recovery (Own) Touchdown") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
          | ((play_df["type.text"] == "Fumble Recovery (Opponent)") & play_df["text"].str.match("pass complete|pass incomplete|pass intercepted", case=False, flags=0, na=False))
           | ((play_df["type.text"] == "Fumble Recovery (Opponent)") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
           | ((play_df["type.text"] == "Fumble Recovery (Opponent) Touchdown") & play_df["text"].str.match("pass complete|pass incomplete", case=False, flags=0, na=False))
           | ((play_df["type.text"] == "Fumble Return Touchdown") & play_df["text"].str.match("pass complete|pass incomplete", case=False, flags=0, na=False))
            | ((play_df["type.text"] == "Fumble Return Touchdown") & play_df["text"].str.match("sacked", case=False, flags=0, na=False))
        ),
        1, 0
    )

      #-- Sacks----
    play_df['sack_vec'] = np.where(
        (
            (play_df["type.text"].isin(["Sack", "Sack Touchdown"]))
            | ((play_df["type.text"].isin(["Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Recovery (Opponent)", "Fumble Recovery (Opponent) Touchdown", "Fumble Return Touchdown"]) 
               & (play_df["pass"] == 1)
               & (play_df["text"].str.match("sacked", case=False, flags=0, na=False))
              ))
        ), 1, 0)
    
    play_df['type.text'] = np.where(play_df["text"].str.match(" coin toss ", case=False, flags=0, na=False), "Coin Toss", play_df["type.text"])
    play_df['change_of_poss'] = np.where(play_df["start.team.id"] == play_df["start.team.id"].shift(-1), 0, 1)
    play_df['change_of_poss'] = np.where(play_df['change_of_poss'].isna(), 0, play_df['change_of_poss'])

      ## Fix Strip-Sacks to Fumbles----
    play_df['type.text'] = np.where(
        (play_df.fumble_vec == 1) 
            & (play_df["pass"] == 1) 
            & (play_df.change_of_poss == 1) 
            & (play_df.td_play == 0) 
            & (play_df["start.down"] != 4) 
            & ~(play_df['type.text'].isin(defense_score_vec)),
        "Fumble Recovery (Opponent)", play_df['type.text']
    )
    
    play_df['type.text'] = np.where(
        (play_df.fumble_vec == 1) 
            & (play_df["pass"] == 1) 
            & (play_df.change_of_poss == 1) 
            & (play_df.td_play == 1),
        "Fumble Recovery (Opponent) Touchdown", play_df['type.text']
    )
  #       ## Fix rushes with fumbles and a change of possession to fumbles----  
    play_df['type.text'] = np.where(
        (play_df.fumble_vec == 1) 
            & (play_df["rush"] == 1) 
            & (play_df.change_of_poss == 1) 
            & (play_df.td_play == 0) 
            & (play_df["start.down"] != 4) 
            & ~(play_df['type.text'].isin(defense_score_vec)),
        "Fumble Recovery (Opponent)", play_df['type.text']
    )

    play_df['type.text'] = np.where(
        (play_df.fumble_vec == 1) 
            & (play_df["rush"] == 1) 
            & (play_df.change_of_poss == 1) 
            & (play_df.td_play == 1),
        "Fumble Recovery (Opponent) Touchdown", play_df['type.text']
    )

#       ## Portion of touchdown check for plays where touchdown is not listed in the play_type--
    play_df["td_check"] = play_df["text"].str.contains("Touchdown")

#       #-- Fix kickoff fumble return TDs ----
    play_df['type.text'] = np.where(
            (play_df.kickoff_play == 1) 
            & (play_df.fumble_vec == 1) 
            & (play_df.td_play == 1)
            & (play_df.td_check == 1),
        f"{play_df['type.text']} Touchdown", play_df['type.text']
    )

#       #-- Fix punt return TDs ----
    play_df['type.text'] = np.where(
            (play_df.punt_play == 1) 
            & (play_df.td_play == 1)
            & (play_df.td_check == 1),
        f"{play_df['type.text']} Touchdown", play_df['type.text']
    )

#       #-- Fix kick return TDs----
    play_df['type.text'] = np.where(
            (play_df.kickoff_play == 1) 
            & (play_df.fumble_vec == 0) 
            & (play_df.td_play == 1)
            & (play_df.td_check == 1),
        f"Kickoff Return Touchdown", play_df['type.text']
    )

#       #-- Fix rush/pass tds that aren't explicit----
#       play_type = ifelse(.data$td_play == 1 & .data$rush == 1 &
#                            .data$fumble_vec == 0 & .data$td_check == 1,
#                          "Rushing Touchdown", 
#                          .data$play_type),
#       play_type = ifelse(.data$td_play == 1 & .data$pass == 1 & .data$td_check == 1 &
#                            .data$fumble_vec == 0 & !(.data$play_type %in% int_vec),
#                          "Passing Touchdown", 
#                          .data$play_type),
#       play_type = ifelse(.data$pass == 1 & .data$play_type == "Pass Reception" &
#                            .data$yards_gained == .data$yards_to_goal &
#                            .data$fumble_vec == 0 & !(.data$play_type %in% int_vec),
#                          "Passing Touchdown", 
#                          .data$play_type),
#       play_type = ifelse(.data$play_type == "Blocked Field Goal" & 
#                            stringr::str_detect(.data$play_text, regex("for a TD", ignore_case = TRUE)),
#                          "Blocked Field Goal Touchdown", 
#                          .data$play_type),

#       #-- Fix duplicated TD play_type labels----
#       play_type = ifelse(.data$play_type == "Punt Touchdown Touchdown", "Punt Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Fumble Return Touchdown Touchdown", "Fumble Return Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Rushing Touchdown Touchdown", "Rushing Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Uncategorized Touchdown Touchdown", "Uncategorized Touchdown", .data$play_type),

#       #-- Fix Pass Interception Return TD play_type labels----
#       play_type = ifelse(stringr::str_detect(.data$play_text, "pass intercepted for a TD") & !is.na(.data$play_text),
#                          "Interception Return Touchdown", .data$play_type),

#       #-- Fix Sack/Fumbles Touchdown play_type labels----
#       play_type = ifelse(stringr::str_detect(.data$play_text, regex("sacked", ignore_case = TRUE)) & 
#                            stringr::str_detect(.data$play_text, regex("fumbled", ignore_case = TRUE)) &
#                            stringr::str_detect(.data$play_text, regex("TD",ignore_case = TRUE)) &
#                            !is.na(.data$play_text), 
#                          "Fumble Recovery (Opponent) Touchdown", .data$play_type),

#       #-- Fix generic pass plays ----
#       ##-- first one looks for complete pass
#       play_type = ifelse(.data$play_type == "Pass" & str_detect(.data$play_text,"pass complete"),
#                          "Pass Completion", .data$play_type),
#       ##-- second one looks for incomplete pass
#       play_type = ifelse(.data$play_type == "Pass" & str_detect(.data$play_text,"pass incomplete"),
#                          "Pass Incompletion", .data$play_type),
#       ##-- third one looks for interceptions 
#       play_type = ifelse(.data$play_type == "Pass" & str_detect(.data$play_text,"pass intercepted"),
#                          "Pass Interception", .data$play_type),
#       ##-- fourth one looks for sacked
#       play_type = ifelse(.data$play_type == "Pass" & str_detect(.data$play_text,"sacked"), "Sack", .data$play_type),
#       ##-- fifth one play type is Passing Touchdown, but its intercepted 
#       play_type = ifelse(.data$play_type == "Passing Touchdown" & str_detect(.data$play_text,"pass intercepted for a TD"),
#                          "Interception Return Touchdown", .data$play_type),
#       #--- Moving non-Touchdown pass interceptions to one play_type: "Interception Return" -----
#       play_type = ifelse(.data$play_type == "Interception", "Interception Return", .data$play_type),
#       play_type = ifelse(.data$play_type == "Pass Interception", "Interception Return", .data$play_type),
#       play_type = ifelse(.data$play_type == "Pass Interception Return", "Interception Return", .data$play_type),
#       #--- Moving Kickoff/Punt Touchdowns without fumbles to Kickoff/Punt Return Touchdown
#       play_type = ifelse(.data$play_type == "Kickoff Touchdown" & .data$fumble_vec == 0, 
#                          "Kickoff Return Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type %in% c("Kickoff", "Kickoff Return (Offense)") & 
#                            .data$fumble_vec == 1 & .data$change_of_pos_team == 1, 
#                          "Kickoff Team Fumble Recovery", .data$play_type),
#       play_type = ifelse(.data$play_type == "Punt Touchdown" & 
#                            (.data$fumble_vec == 0 | (.data$fumble_vec == 1 & .data$game_id == 401112100)),
#                          "Punt Return Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Punt" & .data$fumble_vec == 1 & .data$change_of_poss == 0,
#                          "Punt Team Fumble Recovery", .data$play_type),
#       play_type = ifelse(.data$play_type == "Punt Touchdown", "Punt Team Fumble Recovery Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Kickoff Touchdown", "Kickoff Team Fumble Recovery Touchdown", .data$play_type),
#       play_type = ifelse(.data$play_type == "Fumble Return Touchdown" & (.data$pass == 1|.data$rush == 1), 
#                          "Fumble Recovery (Opponent) Touchdown", .data$play_type),
#       #--- Safeties (kickoff, punt, penalty) ----
#       play_type = ifelse(.data$play_type %in% c("Pass Reception", "Rush", "Rushing Touchdown") & 
#                            (.data$pass == 1|.data$rush == 1) & .data$safety == 1, 
#                          "Safety", .data$play_type),
#       play_type = ifelse(.data$kickoff_safety == 1, "Kickoff (Safety)", .data$play_type),
#       play_type = ifelse(.data$punt_safety == 1,  paste0(.data$play_type, " (Safety)"), .data$play_type),
#       play_type = ifelse(.data$penalty_safety == 1,  paste0(.data$play_type, " (Safety)"), .data$play_type),


#     #-- 'Penalty' in play text ----
#       pen_text = stringr::str_detect(raw_df$play_text, regex("penalty", ignore_case = TRUE))
#       #-- 'Declined' in play text ----
#       pen_declined_text = stringr::str_detect(raw_df$play_text, regex("declined", ignore_case = TRUE))
#       #-- 'No Play' in play text ----
#       pen_no_play_text = stringr::str_detect(raw_df$play_text, regex("no play", ignore_case = TRUE))
#       #-- 'Off-setting' in play text ----
#       pen_offset_text = stringr::str_detect(raw_df$play_text, regex("off-setting", ignore_case = TRUE))
#       #-- '1st Down' in play text ----
#       pen_1st_down_text = stringr::str_detect(raw_df$play_text, regex("1st down", ignore_case = TRUE))

#       #-- Penalty play_types
#       pen_type = raw_df$play_type == "Penalty" | raw_df$play_type == "penalty"

#       #-- T/F flag conditions penalty_flag 
#       raw_df$penalty_flag = FALSE
#       raw_df$penalty_flag[pen_type] <- TRUE
#       raw_df$penalty_flag[pen_text] <- TRUE

#       #-- T/F flag conditions penalty_declined 
#       raw_df$penalty_declined = FALSE
#       raw_df$penalty_declined[pen_text & pen_declined_text] <- TRUE
#       raw_df$penalty_declined[pen_type & pen_declined_text] <- TRUE

#       #-- T/F flag conditions penalty_no_play 
#       raw_df$penalty_no_play = FALSE
#       raw_df$penalty_no_play[pen_text & pen_no_play_text] <- TRUE
#       raw_df$penalty_no_play[pen_type & pen_no_play_text] <- TRUE

#       #-- T/F flag conditions penalty_offset 
#       raw_df$penalty_offset = FALSE
#       raw_df$penalty_offset[pen_text & pen_offset_text] <- TRUE
#       raw_df$penalty_offset[pen_type & pen_offset_text] <- TRUE

#       #-- T/F flag conditions penalty_1st_conv 
#       raw_df$penalty_1st_conv = FALSE
#       raw_df$penalty_1st_conv[pen_text & pen_1st_down_text] <- TRUE
#       raw_df$penalty_1st_conv[pen_type & pen_1st_down_text] <- TRUE

#       #-- T/F flag for penalty text but not penalty play type -- 
#       raw_df$penalty_text <- FALSE
#       raw_df$penalty_text[pen_text & !pen_type & !pen_declined_text & 
#                             !pen_offset_text & !pen_no_play_text] <- TRUE

#       #--- Sacks ----
    play_df['sack'] = ((play_df['type.text'].isin(["Sack"]) 
                           | (play_df['type.text'].isin(["Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Recovery (Opponent)", "Fumble Recovery (Opponent) Touchdown"]) & play_df['pass'] == 1 & play_df["text"].str.match("sacked", case=False, flags=0, na=False)))
                           | (play_df['type.text'].isin(["Safety"]) & play_df["text"].str.match("sacked", case=False, flags=0, na=False)))

#       #--- Interceptions ------
    play_df["int"] = play_df["type.text"].isin(["Interception Return", "Interception Return Touchdown"])
    play_df["int_td"] = play_df["type.text"].isin(["Interception Return Touchdown"])

#       #--- Pass Completions, Attempts and Targets -------
    play_df['completion'] = ((play_df['type.text'].isin(["Pass Reception", "Pass Completion", "Passing Touchdown"])) 
                           | (play_df['type.text'].isin(["Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Recovery (Opponent)", "Fumble Recovery (Opponent) Touchdown"]) & play_df['pass'] == 1 & ~play_df["text"].str.match("sacked", case=False, flags=0, na=False)))

    play_df['pass_attempt'] = ((play_df['type.text'].isin(["Pass Reception", "Pass Completion", "Passing Touchdown", "Pass Incompletion"])) 
                           | (play_df['type.text'].isin(["Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Recovery (Opponent)", "Fumble Recovery (Opponent) Touchdown"]) & play_df['pass'] == 1 & ~play_df["text"].str.match("sacked", case=False, flags=0, na=False)))

    play_df['target'] = ((play_df['type.text'].isin(["Pass Reception", "Pass Completion", "Passing Touchdown", "Pass Incompletion"])) 
                           | (play_df['type.text'].isin(["Fumble Recovery (Own)", "Fumble Recovery (Own) Touchdown", "Fumble Recovery (Opponent)", "Fumble Recovery (Opponent) Touchdown"]) & play_df['pass'] == 1 & ~play_df["text"].str.match("sacked", case=False, flags=0, na=False)))

#       #--- Pass/Rush TDs ------
    play_df['pass_td'] = (play_df["type.text"] == "Passing Touchdown")
    play_df['rush_td'] = (play_df["type.text"] == "Rushing Touchdown")
    
#       #-- Change of possession via turnover
    play_df['turnover_vec'] = play_df["type.text"].isin(turnover_vec)
    play_df['offense_score_play'] = play_df["type.text"].isin(offense_score_vec)
    play_df['defense_score_play'] = play_df["type.text"].isin(defense_score_vec)
#     play_df['downs_turnover'] = np.where(
#       (play_df["type.text"].isin(normalplay))
#       & (play_df["statYardage"] < play_df["start.distance"])
#       & (play_df["start.down"] == 4)
#       & (play_df["penalty_1st_conv"] != 1)
#     , 1, 0)

#       #-- Touchdowns----
    play_df['scoring_play'] = play_df["type.text"].isin(scores_vec)
    play_df['yds_punted'] = play_df["text"].str.extract("(?<= punt for)[^,]+(\d+)", flags=re.IGNORECASE).astype(float)
    play_df['yds_punt_gained'] = np.where(play_df.punt == 1, play_df["statYardage"], None)
    play_df['fg_inds'] = play_df["type.text"].str.match("Field Goal", case=False, flags=0, na=False)
    play_df['fg_made'] = (play_df["type.text"] == "Field Goal Good")
    play_df['yds_fg'] = play_df["text"].str.extract("(\d{0,2}) Yd|Yard FG|Field", flags=re.IGNORECASE).astype(float)
    play_df['start.yardsToEndzone'] = np.where(play_df['fg_inds'] == True, play_df['yds_fg'] - 17, play_df["start.yardsToEndzone"])

    play_df["pos_unit"] = np.select(
        [
            play_df.punt == 1, 
            play_df.kickoff_play == 1,
            play_df.fg_inds == True,
            play_df["type.text"] == "Defensive 2pt Conversion"
        ], 
        [
            'Punt Offense', 
            'Kickoff Return', 
            'Field Goal Offense',
            'Offense'
        ], 
        default='Offense'
    )
    
    play_df["def_pos_unit"] = np.select(
        [
            play_df.punt == 1, 
            play_df.kickoff_play == 1,
            play_df.fg_inds == True,
            play_df["type.text"] == "Defensive 2pt Conversion"
        ], 
        [
            'Punt Return', 
            'Kickoff Defense', 
            'Field Goal Defense',
            'Defense'
        ], 
        default='Defense'
    )
#       #--- Lags/Leads play type ----
    play_df['lag_play_type3'] = play_df['type.text'].shift(3)
    play_df['lag_play_type2'] = play_df['type.text'].shift(2)
    play_df['lag_play_type'] = play_df['type.text'].shift(1)
    
    play_df['lead_play_type'] = play_df['type.text'].shift(-1)
    play_df['lead_play_type2'] = play_df['type.text'].shift(-2)
    play_df['lead_play_type3'] = play_df['type.text'].shift(-3)

    return play_df

In [118]:
clean_pbp_dat(plays_json)

,period,homeScore,scoringPlay,priority,statYardage,awayScore,wallclock,modified,id,text,...,fg_made,yds_fg,pos_unit,def_pos_unit,lag_play_type3,lag_play_type2,lag_play_type,lead_play_type,lead_play_type2,lead_play_type3
0,1,0,False,False,0,0,2020-12-21T19:34:38Z,2020-12-21T19:35Z,401256135101849902,Zach Williams kickoff for 60 yds,...,False,60.0,Kickoff Return,Kickoff Defense,NaN,NaN,NaN,Rush,Pass Incompletion,Rush
1,1,0,False,False,-2,0,2020-12-21T19:36:09Z,2020-12-21T19:36Z,401256135101855901,Camerun Peoples run for a loss of 2 yards to t...,...,False,NaN,Offense,Defense,NaN,NaN,Kickoff,Pass Incompletion,Rush,Punt
2,1,0,False,False,0,0,2020-12-21T19:36:29Z,2020-12-21T19:36Z,401256135101858101,Zac Thomas pass incomplete to Jalen Virgil,...,False,NaN,Offense,Defense,NaN,Kickoff,Rush,Rush,Punt,Rush
3,1,0,False,False,4,0,2020-12-21T19:37:31Z,2020-12-21T19:37Z,401256135101864601,Zac Thomas run for 4 yds to the AppSt 37,...,False,4.0,Offense,Defense,Kickoff,Rush,Pass Incompletion,Punt,Rush,Rush
4,1,0,False,False,0,0,2020-12-21T19:38:31Z,2020-12-21T19:38Z,401256135101867201,"Xavier Subotsch punt for 29 yds, downed at the...",...,False,29.0,Punt Offense,Punt Return,Rush,Pass Incompletion,Rush,Rush,Rush,Pass Incompletion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,4,49,False,False,16,23,2020-12-23T03:23:40Z,2020-12-23T03:23Z,401256137104979401,Zach Wilson pass complete to Dax Milne for 16 ...,...,False,16.0,Offense,Defense,Rush,Rush,Pass Incompletion,Timeout,Rush,Rush
375,4,49,False,False,0,23,2020-12-23T03:24:04Z,2020-12-23T03:24Z,401256137104986501,"Timeout UCF, clock 01:34",...,False,NaN,Offense,Defense,Rush,Pass Incompletion,Pass Reception,Rush,Rush,Rush
376,4,49,False,False,-2,23,2020-12-23T03:28:13Z,2020-12-23T03:28Z,401256137104986901,TEAM run for a loss of 2 yards to the UCF 6,...,False,NaN,Offense,Defense,Pass Incompletion,Pass Reception,Timeout,Rush,Rush,NaN
377,4,49,False,False,-1,23,2020-12-23T03:28:45Z,2020-12-23T03:28Z,401256137104994401,TEAM run for a loss of 1 yard to the UCF 7,...,False,NaN,Offense,Defense,Pass Reception,Timeout,Rush,Rush,NaN,NaN
